In [1]:
import os
import json
from openai import OpenAI
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv

load_dotenv()

# Initialize the Azure OpenAI client
client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_WEBSEARCH_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Define the deployment you want to use for your chat completions API calls

deployment_name = "gpt-4o-mini"

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo" : "Asia/Tokyo",
    "san francisco" : "America/Los_Angeles",
    "paris" : "Europe/Paris",
    "seoul" : "Asia/Seoul"
}

def get_current_time(location):
    """주어진 지역들의 현재 시각을 알려줘"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

def run_conversation():
    # Initial user message
    messages = [{"role": "user", "content": "현재 서울과 샌프란시스코의 시간을 알려줘"}] # Single function call
    #messages = [{"role": "user", "content": "What's the current time in San Francisco, Tokyo, and Paris?"}] # Parallel function call with a single tool/function defined

    # Define the function for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the function
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                time_response = get_current_time(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content

# Run the conversation and print the result
print(run_conversation())

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_bh15e2oTwHhlL41tuTc1cs2J', function=Function(arguments='{"location": "Seoul"}', name='get_current_time'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_uDc80LsduEGaYvDkCv26idG9', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'Seoul'}
get_current_time called with location: Seoul
Timezone found for seoul
Function arguments: {'location': 'San Francisco'}
get_current_time called with location: San Francisco
Timezone found for san francisco
현재 서울은 오전 10시 53분이고, 샌프란시스코는 오후 5시 53분입니다.


In [ ]:
# ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, 
#                       tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_c4JGldQJvR0UP10OcxOigBL7', 
#                                                                         function=Function(arguments='{"location": "Seoul"}', 
#                                                                                           name='get_current_time'), 
#                                                                                           type='function'), 
#                                   ChatCompletionMessageFunctionToolCall(id='call_HLISlj8cIVvAQBcrvrM0YenH', 
#                                                           function=Function(arguments='{"location": "San Francisco"}',
#                                                                              name='get_current_time'),
#                                                                              type='function')])
# Function arguments: {'location': 'Seoul'}